In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/data_hes_quotes_france_202512-2.csv')
df.head()

import pandas as pd
import numpy as np

print("="*80)
print("DATA CLEANING & PREPARATION")
print("="*80)

# Assuming df is already loaded from your CSV
print(f"Initial data loaded. Shape: {df.shape}")
print(f"Columns: {list(df.columns)[:10]}...")  # Show first 10 columns

# 1. Identify test accounts
print("\n🔍 IDENTIFYING TEST/TRAINING ACCOUNTS...")

def identify_test_accounts(df):
    """Identify suspicious accounts for removal"""
    
    # Known test account from discovery
    known_test = ['CL00201682']
    
    # Pattern 1: High volume, zero conversion
    print("  Analyzing high-volume, zero-conversion accounts...")
    customer_stats = df.groupby('numero_compte').agg({
        'fg_devis_accepte': ['count', 'mean']
    })
    customer_stats.columns = ['quote_count', 'conversion_rate']
    
    # Find accounts with >20 quotes and 0% conversion
    high_volume_zero = customer_stats[
        (customer_stats['quote_count'] > 20) & 
        (customer_stats['conversion_rate'] == 0)
    ].sort_values('quote_count', ascending=False)
    
    print(f"  Found {len(high_volume_zero)} accounts with >20 quotes and 0% conversion")
    if len(high_volume_zero) > 0:
        print(f"  Top 5 by volume: {high_volume_zero.index[:5].tolist()}")
    
    # Pattern 2: Unrealistic quote frequencies (>1 quote per day average)
    print("\n  Analyzing quote frequency patterns...")
    df['dt_creation_devis'] = pd.to_datetime(df['dt_creation_devis'])
    
    customer_timelines = df.groupby('numero_compte').agg({
        'dt_creation_devis': ['min', 'max', 'count']
    })
    customer_timelines.columns = ['first_quote', 'last_quote', 'quote_count']
    
    customer_timelines['days_active'] = (
        customer_timelines['last_quote'] - customer_timelines['first_quote']
    ).dt.days + 1
    
    # Avoid division by zero for single-quote customers
    customer_timelines['quotes_per_day'] = np.where(
        customer_timelines['days_active'] > 0,
        customer_timelines['quote_count'] / customer_timelines['days_active'],
        1  # Single quote on one day = 1 quote/day
    )
    
    unrealistic_frequency = customer_timelines[
        customer_timelines['quotes_per_day'] > 1
    ].sort_values('quotes_per_day', ascending=False)
    
    print(f"  Found {len(unrealistic_frequency)} accounts with >1 quote/day average")
    if len(unrealistic_frequency) > 0:
        print(f"  Most extreme: {unrealistic_frequency.index[0]} with {unrealistic_frequency.iloc[0]['quotes_per_day']:.1f} quotes/day")
    
    # Pattern 3: Employees/Internal (simplified - would need HR data in reality)
    print("\n  Checking for potential employee accounts...")
    # This is a placeholder - in reality would cross-reference with HR database
    # For now, look for patterns like company email domains, known employee names
    employee_patterns = []
    
    # Combine all suspicious accounts
    all_suspicious = list(set(
        known_test + 
        high_volume_zero.index.tolist()[:10] +  # Top 10 high-volume, zero-conversion
        unrealistic_frequency.index.tolist()[:5]  # Top 5 by frequency
    ))
    
    return all_suspicious, high_volume_zero, unrealistic_frequency

# Run test account identification
test_accounts, high_volume_zero, unrealistic_frequency = identify_test_accounts(df)

print(f"\n📊 TEST ACCOUNT ANALYSIS:")
print(f"Total suspicious accounts identified: {len(test_accounts)}")
print(f"Known test account (CL00201682): 229 quotes, 0% conversion")

# Show impact of each pattern
if len(high_volume_zero) > 0:
    top_high_volume = high_volume_zero.iloc[0]
    print(f"Highest volume zero-conversion: {high_volume_zero.index[0]} with {int(top_high_volume['quote_count'])} quotes")

if len(unrealistic_frequency) > 0:
    top_frequency = unrealistic_frequency.iloc[0]
    print(f"Highest frequency: {unrealistic_frequency.index[0]} with {top_frequency['quotes_per_day']:.1f} quotes/day")

# 2. Remove test accounts and create clean dataset
print("\n🧹 CREATING CLEAN DATASET...")
df_clean = df[~df['numero_compte'].isin(test_accounts)].copy()
quotes_removed = len(df) - len(df_clean)
percent_removed = (quotes_removed / len(df)) * 100

print(f"Original dataset: {len(df):,} quotes")
print(f"Clean dataset: {len(df_clean):,} quotes")
print(f"Quotes removed: {quotes_removed:,} ({percent_removed:.1f}%)")

# 3. Calculate baseline statistics
print("\n📈 BASELINE STATISTICS (CLEAN DATA):")

# Basic metrics
total_customers = df_clean['numero_compte'].nunique()
total_quotes = len(df_clean)
quotes_per_customer = total_quotes / total_customers
conversion_rate = df_clean['fg_devis_accepte'].mean()

print(f"Total unique customers: {total_customers:,}")
print(f"Total quotes: {total_quotes:,}")
print(f"Avg quotes per customer: {quotes_per_customer:.2f}")
print(f"Quote-level conversion rate: {conversion_rate:.1%}")

# Customer-level conversion
customer_conversion = df_clean.groupby('numero_compte')['fg_devis_accepte'].max()
customer_conversion_rate = customer_conversion.mean()
customers_with_sales = customer_conversion.sum()

print(f"\nCustomer-level metrics:")
print(f"Customers with at least one sale: {customers_with_sales:,} ({customer_conversion_rate:.1%})")
print(f"Customers with no sales: {total_customers - customers_with_sales:,} ({1 - customer_conversion_rate:.1%})")

# Distribution analysis
print("\n📊 DISTRIBUTION ANALYSIS:")
quote_counts = df_clean['numero_compte'].value_counts()
print(f"Customers with 1 quote: {(quote_counts == 1).sum():,} ({(quote_counts == 1).sum()/total_customers:.1%})")
print(f"Customers with 2-3 quotes: {((quote_counts >= 2) & (quote_counts <= 3)).sum():,} ({((quote_counts >= 2) & (quote_counts <= 3)).sum()/total_customers:.1%})")
print(f"Customers with 4+ quotes: {(quote_counts >= 4).sum():,} ({(quote_counts >= 4).sum()/total_customers:.1%})")

# Temporal analysis
print("\n📅 TEMPORAL ANALYSIS:")
df_clean['dt_creation_devis'] = pd.to_datetime(df_clean['dt_creation_devis'])
print(f"Date range: {df_clean['dt_creation_devis'].min().date()} to {df_clean['dt_creation_devis'].max().date()}")

# Check data quality issues
print("\n🔍 DATA QUALITY CHECK:")
missing_target = df_clean['fg_devis_accepte'].isna().sum()
print(f"Missing target values: {missing_target} ({missing_target/len(df_clean):.1%})")

# Remove any remaining NaN in target (if any)
if missing_target > 0:
    df_clean = df_clean.dropna(subset=['fg_devis_accepte'])
    print(f"Removed {missing_target} rows with NaN target")

# 4. Save clean dataset for Day 2
output_file = 'cleaned_quote_data.csv'
df_clean.to_csv(output_file, index=False)
print(f"\n💾 Clean dataset saved to: {output_file}")


DATA CLEANING & PREPARATION
Initial data loaded. Shape: (38697, 37)
Columns: ['id_devis', 'num_devis', 'nom_devis', 'nom_agence', 'nom_filiale_zone', 'nom_region', 'statut_devis', 'fg_devis_emis', 'fg_devis_refuse', 'fg_devis_accepte']...

🔍 IDENTIFYING TEST/TRAINING ACCOUNTS...
  Analyzing high-volume, zero-conversion accounts...
  Found 4 accounts with >20 quotes and 0% conversion
  Top 5 by volume: ['CL00201682', 'CL00069020', 'CL00291534', 'CL00294368']

  Analyzing quote frequency patterns...
  Found 3734 accounts with >1 quote/day average
  Most extreme: CL00271519 with 6.0 quotes/day

  Checking for potential employee accounts...

📊 TEST ACCOUNT ANALYSIS:
Total suspicious accounts identified: 9
Known test account (CL00201682): 229 quotes, 0% conversion
Highest volume zero-conversion: CL00201682 with 227 quotes
Highest frequency: CL00271519 with 6.0 quotes/day

🧹 CREATING CLEAN DATASET...
Original dataset: 38,697 quotes
Clean dataset: 38,342 quotes
Quotes removed: 355 (0.9%)

📈 B